# Model to accurately forecast inventory demand based on historical sales data. 

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import time
import random
import pickle

## Load Data 
The size of the training data is quite large (~4 GB). Large datasets require significant amount of memory to process. Instead, we will sample the data randomly for our initial data analysis and visualization. 

In [2]:
def load_samp_data(filename='train.csv', columns=[], load_pkl=1):
    """ 
      Function returns a dataframe containing the training data sampled randomly. 
      The data is also stored in a pickle file for later processing.
    """
    if load_pkl:
        inputfile = open('train_samp_data.pkl', 'rb')
        data = pickle.load(inputfile)
        inputfile.close()
        return data
    
    chunksize= 10 ** 6
    datasize = 74180464 #datasize = sum(1 for line in open(filename)) - 1 #number of records in file (excludes header)
    samplesize = 10 ** 3 # samples per chunk of data read from the file.
    
    data = pd.DataFrame([],columns=columns)
    chunks = pd.read_csv(filename, iterator=True, chunksize=chunksize)
    for chunk in chunks:
        chunk.columns = columns
        data = data.append(chunk.sample(samplesize)) 
    
    # write data to a pickle file.
    outputfile = open('train_samp_data.pkl','wb')
    pickle.dump(data,outputfile)
    outputfile.close()
    
    return data
 
load_pkl = 1
columns = ['week_num', 'sales_depot_id', 'sales_chan_id', 'route_id', 'client_id', 'prod_id', 'saleunit_curr_wk', 'saleamt_curr_wk', 'retunit_next_week', 'retamt_next_wk', 'y_pred_demand']
tic = time.time()
train_data_samp = load_samp_data('train.csv', columns, load_pkl)
toc = time.time()
print '**'
print 'Time to load: ', toc-tic, 'sec'
print 
print train_data_samp.describe()
print '**'

**
Time to load:  0.146552801132 sec

           week_num  sales_depot_id  sales_chan_id      route_id  \
count  75000.000000    75000.000000   75000.000000  75000.000000   
mean       5.983653     2776.219267       1.377787   2111.400187   
std        2.027215     4644.927230       1.450673   1480.490670   
min        3.000000     1110.000000       1.000000      1.000000   
25%        4.000000     1312.000000       1.000000   1161.000000   
50%        6.000000     1614.000000       1.000000   1285.000000   
75%        8.000000     2038.000000       1.000000   2802.000000   
max        9.000000    25759.000000      11.000000   9319.000000   

          client_id       prod_id  saleunit_curr_wk  saleamt_curr_wk  \
count  7.500000e+04  75000.000000      75000.000000     75000.000000   
mean   1.795300e+06  20893.475467          7.204680        67.336995   
std    1.832930e+06  18660.676981         21.827121       290.845496   
min    1.060000e+02     72.000000          0.000000         0

In [3]:
clientnameid_data = pd.read_csv('cliente_tabla.csv')
clientnameid_data.head()

,Cliente_ID,NombreCliente
0,0,SIN NOMBRE
1,1,OXXO XINANTECATL
2,2,SIN NOMBRE
3,3,EL MORENO
4,4,SDN SER DE ALIM CUERPO SA CIA DE INT


## Data Cleaning
There are duplicate client ids in cliente_table, which means one client id may have multiple client name that are very similar. We will cluster them based on a hash function and use a clustering algorithm to evaluate similarity.  

In [4]:
def hash_eval(s):
    hash_base = 4
    s = "".join(s.split())
    seqlen = len(s)
    n = seqlen - 1
    h = 0
    for c in s:
        h += ord(c) * (hash_base ** n)
        n -= 1
    curhash = h
    return curhash

clientid_hash = dict()
new_client_id = [-1]   
for idx, s in enumerate(clientnameid_data.NombreCliente):
    t = hash_eval(s)
    clientid_hash.setdefault(hash_eval(s), []).append(clientnameid_data.Cliente_ID[idx])
    if t in clientid_hash:
        a = clientid_hash[t]
        new_client_id.append(a[0])
    else: 
        new_client_id.append(np.max(new_client_id)+1)


In [14]:
clientnameid_data['New_Cliente_ID'] = new_client_id[1:]

Cliente_ID       935362
NombreCliente    935362
dtype: int64 935362


In [16]:
clientnameid_data.head(10)

#print len(clientid_hash), ' ', len(np.unique(np.array(clientid_hash)))
#sns.jointplot(np.arange(len(new_client_ID)), np.array(new_client_ID))
#sns.jointplot(np.arange([len(new_client_ID)]), np.array(new_client_ID))

,Cliente_ID,NombreCliente,New_Cliente_ID
0,0,SIN NOMBRE,0
1,1,OXXO XINANTECATL,1
2,2,SIN NOMBRE,0
3,3,EL MORENO,3
4,4,SDN SER DE ALIM CUERPO SA CIA DE INT,4
5,4,SDN SER DE ALIM CUERPO SA CIA DE INT,4
6,5,LA VAQUITA,5
7,6,LUPITA,6
8,7,I M EL GUERO,7
9,8,MINI SUPER LOS LUPES,8


Large datasets require significant amount of memory. Instead of loading the entire training data into memory, we use a subset of training data which is sampled uniformly at random for our initial data analysis and visualization. 

In [ ]:
startt = time.time()
datasize = sum(1 for line in open(filename)) - 1 #number of records in file (excludes header)
endt = time.time()
print endt-startt

In [ ]:
datasize = 74180464
samplesize = 5000


data_train.columns = ['week_num', 'sales_depot_id', 'sales_chan_id', 'route_id', 'client id', 'prod_id', 'saleunit_curr_wk', 'saleamt_curr_wk', 'retunit_next_week', 'retamt_next_wk', 'y_pred_demand']
data_test = pd.read_csv('test.csv', nrows=100)
data_test.columns = ['id', 'week_num', 'sales_depot_id', 'sales_chan_id', 'route_id', 'client id', 'prod_id']
data_test.head()

In [ ]:
g = sns.PairGrid(data_t)
g.map(plt.scatter)

In [ ]:
chunksize= 10 ** 6
datasize = 0
startt = time.time()
for chunk in pd.read_csv(filename, chunksize=chunksize):
    datasize = datasize + len(chunk)
    

    
endt = time.time()
del chunk
print 'time: ', endt-startt, '(sec), datasize: ', datasize

datasize = sum(1 for line in open(filename)) - 1 #number of records in file (excludes header)
s = 10000 #desired sample size
skip = sorted(random.sample(xrange(1,n+1),n-s)) #the 0-indexed header will not be included in the skip list
df = pandas.read_csv(filename, skiprows=skip)


#sns.distplot(train_data_samp.client_id)
#sns.stripplot(x="week_num", y="saleunit_curr_wk", data=train_data_samp, jitter=True);
#sns.jointplot(x='sales_depot_id', y='saleunit_curr_wk', data=train_data_samp)
#sns.jointplot(x='prod_id', y='saleunit_curr_wk', data=train_data_samp)
#g = sns.PairGrid(train_data_samp)
#g.map(plt.scatter)






In [ ]:
Feature Analysis: Investigate the feature vector space.